<a href="https://colab.research.google.com/github/Onmang/Multimedia_engineering/blob/master/t20me022_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

期末レポート-音声単語35種類の分類

T20ME022 グエンコンフィ

課題と一緒に提示された参考コードに改良を施し、分類精度の向上に努める。
行った試行を次に示す。


0.   参考コードをそのまま実行し、分類精度の確認を行った。


> epochs:50,  結果：61.0%

> epochs:100,  結果：63.4%

  
> エポック数ではこのモデルの分類精度を上げることは難しいそうである。





1.   
2.   リスト項目




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 51.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0

In [3]:
import tensorflow
tensorflow.__version__

'2.10.0'

In [7]:
import pathlib
import matplotlib.pyplot as plt # グラフを書くためのライブラリ
import numpy as np # NumPy
import tensorflow as tf
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences # パディング用
from scipy.io import wavfile # wavファイル読み込み用
import librosa # MFCC計算用
import glob # ファイル一覧を取得するためのライブラリ
import os


In [ ]:
!unzip /content/drive/MyDrive/speech_commands.zip

In [8]:
# ファイルリストから音声ファイルの一覧を作成する
train_listfile = 'speech_commands/00_train.txt'
valid_listfile = 'speech_commands/01_valid.txt'
test_listfile = 'speech_commands/02_test.txt'

def get_filelist(list):
  with open(list) as f:
    lines = f.readlines()
  return [os.path.join('speech_commands', l.strip()) for l in lines]

train_list = get_filelist(train_listfile)
valid_list = get_filelist(valid_listfile)
test_list = get_filelist(test_listfile)

In [9]:
# ラベル作成（アルファベット順でソート）
labels = sorted([d for d in os.listdir('speech_commands') if os.path.isdir(os.path.join('speech_commands', d))]) # ラベル名を取得する
# ラベルの確認
print(labels)


['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']


In [10]:
# ファイル名からラベル情報を取得する関数
def get_label(filename):
  cls = os.path.dirname(filename).split('/')[1]
  return labels.index(cls)

# ミニバッチデータの作成
def make_batch(data_files):
  batch_data = [] # データを入れるリスト
  label_data = [] # データに対するラベルを入れるリスト
  ### 1つずつファイルを処理
  for d in data_files:
    filename = bytes.decode(d.numpy()) # ファイル名の取得
    sr, data = wavfile.read(filename) # wavファイルの読み込み
    mels = librosa.feature.melspectrogram(y=np.float32(data), sr=sr, n_mels=64, n_fft=512, hop_length=160) #メルスペクトログラムの計算
    mels = (mels - np.mean(mels)) / np.std(mels) # データを平均0, 分散1になるように正規化
    batch_data.append(mels.T) # 転置してデータをリストに追加
    classid = get_label(filename) # 数字クラスのラベルIDを取得
    label_data.append(classid)
  # バッチデータの長さを揃えるため，短いデータの後ろにゼロを埋める。
  batch_data = pad_sequences(batch_data, padding='post', value=0, dtype=np.float32)
  return batch_data, label_data

batch_size = 50 # ミニバッチサイズ
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.list_files(train_list) # 訓練のファイルリストをTensor型に変換
train_ds = train_ds.shuffle(buffer_size=len(train_list)) # 完全シャッフル
train_ds = train_ds.repeat(1) # 繰り返し
train_ds = train_ds.batch(batch_size) # ミニバッチ生成
valid_ds = tf.data.Dataset.list_files(valid_list) # 検証用のファイルリストをTensor型に変換
valid_ds = valid_ds.batch(batch_size) # ミニバッチ生成
test_ds = tf.data.Dataset.list_files(test_list) # テストのファイルリストをTensor型に変換
test_ds = test_ds.batch(500) # 全てのデータを一括処理する

# ミニバッチ単位で，インデックスから訓練データに変換
train_ds = train_ds.map(lambda x: tf.py_function(make_batch, [x], Tout=[tf.float32, tf.int32]))
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # プレフェッチの設定（実行速度アップのため）
# 検証データのミニバッチ化
valid_ds = valid_ds.map(lambda x: tf.py_function(make_batch, [x], Tout=[tf.float32, tf.int32]))
# テストデータの準備
test_ds = test_ds.map(lambda x: tf.py_function(make_batch, [x], Tout=[tf.float32, tf.int32]))

モデルの定義（授業で扱ったモデル）


In [11]:
# モデル定義
input = Input(shape=(None, 64), name='input') # 入力データの長さ（時間）は未定で，各時刻のデータは64次元
# マスク層。全次元が0のデータを学習時に無視するようにするための層
h = Masking(mask_value=0, input_shape=(None, 64))(input)
h = Bidirectional(LSTM(256, name='lstm1'))(h)
h = Dense(128, activation='relu', name='fc1')(h)
output = Dense(35, activation='softmax', name='output')(h)
model = tf.keras.Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None, 64)]        0         
                                                                 
 masking (Masking)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 512)              657408    
 l)                                                              
                                                                 
 fc1 (Dense)                 (None, 128)               65664     
                                                                 
 output (Dense)              (None, 35)                4515      
                                                                 
Total params: 727,587
Trainable params: 727,587
Non-trainable params: 0
_______________________________________________________

In [12]:
# オプティマイザの設定
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [17]:
# 訓練
model.fit(train_ds, epochs=100, validation_data=valid_ds, use_multiprocessing=True)


Epoch 1/100
100/100 [==============================] - 34s 339ms/step - loss: 0.0112 - sparse_categorical_accuracy: 0.9978 - val_loss: 2.4149 - val_sparse_categorical_accuracy: 0.6100
Epoch 2/100
100/100 [==============================] - 34s 345ms/step - loss: 0.0067 - sparse_categorical_accuracy: 0.9980 - val_loss: 3.8008 - val_sparse_categorical_accuracy: 0.4600
Epoch 3/100
100/100 [==============================] - 34s 345ms/step - loss: 0.0101 - sparse_categorical_accuracy: 0.9972 - val_loss: 2.8748 - val_sparse_categorical_accuracy: 0.5680
Epoch 4/100
100/100 [==============================] - 35s 348ms/step - loss: 0.0076 - sparse_categorical_accuracy: 0.9974 - val_loss: 4.4609 - val_sparse_categorical_accuracy: 0.4000
Epoch 5/100
100/100 [==============================] - 34s 342ms/step - loss: 0.0059 - sparse_categorical_accuracy: 0.9982 - val_loss: 2.9306 - val_sparse_categorical_accuracy: 0.5600
Epoch 6/100
100/100 [==============================] - 35s 348ms/step - loss: 0.

In [16]:
# モデルの保存（訓練条件に応じてまめに保存しておくと良い）
model.save('/content/drive/MyDrive/Final')


テストデータに対する評価

In [18]:
# テストデータの評価コード（これが50％以上）
loss, acc = model.evaluate(test_ds)
print(f'分類精度: {acc*100:.1f}%')


1/1 [==============================] - 2s 2s/step - loss: 2.5430 - sparse_categorical_accuracy: 0.6340
分類精度: 63.4%
